# Xây dựng mô hình mạng Neural đầy đủ

## 1. Tải dữ liệu

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import os

In [36]:
training_data = datasets.FashionMNIST(root='./data', train=True, download=True , transform=ToTensor())
testing_data = datasets.FashionMNIST(root='./data', train=False, download=True , transform=ToTensor())

100.0%
100.0%
100.0%
100.0%


In [37]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [38]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()

In [39]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [40]:
# Chọn loss function
loss_fn = nn.CrossEntropyLoss()

In [41]:
# Chọn bộ tối ưu hóa SGD
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300041  [   64/60000]
loss: 2.287289  [ 6464/60000]
loss: 2.271512  [12864/60000]
loss: 2.256353  [19264/60000]
loss: 2.255076  [25664/60000]
loss: 2.212679  [32064/60000]
loss: 2.204983  [38464/60000]
loss: 2.192146  [44864/60000]
loss: 2.195661  [51264/60000]
loss: 2.153572  [57664/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.151189 

Epoch 2
-------------------------------
loss: 2.128464  [   64/60000]
loss: 2.148231  [ 6464/60000]
loss: 2.079188  [12864/60000]
loss: 2.106180  [19264/60000]
loss: 2.063033  [25664/60000]
loss: 2.063523  [32064/60000]
loss: 1.971497  [38464/60000]
loss: 1.914664  [44864/60000]
loss: 1.949022  [51264/60000]
loss: 1.889342  [57664/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 1.877857 

Epoch 3
-------------------------------
loss: 1.851866  [   64/60000]
loss: 1.898569  [ 6464/60000]
loss: 1.805988  [12864/60000]
loss: 1.855341  [19264/60000]
loss: 1.706504  [25664/60000]
loss: 1.597916  [32064/600